In [1]:
!pip install -U portalocker>=2.0.0

In [2]:
from google.colab import files
import torch

uploaded = files.upload()

Saving PedalBoardData.csv to PedalBoardData.csv


In [3]:
import pandas as pd
import io
import random
from ast import literal_eval
from sklearn.model_selection import train_test_split

#import torch
#import numpy as np
from torchdata.datapipes.iter import ShardingFilter

df = pd.read_csv(io.BytesIO(uploaded['PedalBoardData.csv']))
tuples_list = [list(x) for x in df.to_numpy()]
for idx, data in enumerate(tuples_list):
        tuples_list[idx][0] = literal_eval(data[0])

#tuples_list=np.vstack(tuples_list).astype(np.float)
#tuples_list = torch.from_numpy(tuples_list)
#tuples_list = [item.detach().numpy() for item in df]
train_data , test_data = train_test_split(tuples_list,test_size=0.5)
train_iter = ShardingFilter(train_data)
test_iter = ShardingFilter(test_data)
print(train_iter)


ShardingFilterIterDataPipe


In [4]:
N = len(tuples_list)
positives = [0,0,0,0,0,0,0,0,0]

for row in tuples_list:
    for i, pedal in enumerate(row[0]):
        positives[i] += pedal

print(positives)

class_weights = {}
positive_weights = {}
negative_weights = {}

for i, pedal in enumerate(positives):
    positive_weights[i] = N /(2 * pedal)
    negative_weights[i] = N /(2 * N-pedal)

class_weights['positive_weights'] = positive_weights
class_weights['negative_weights'] = negative_weights
class_weights

[88, 72, 83, 80, 85, 79, 78, 78, 77]


{'positive_weights': {0: 4.534090909090909,
  1: 5.541666666666667,
  2: 4.807228915662651,
  3: 4.9875,
  4: 4.694117647058824,
  5: 5.050632911392405,
  6: 5.115384615384615,
  7: 5.115384615384615,
  8: 5.181818181818182},
 'negative_weights': {0: 0.5291777188328912,
  1: 0.5236220472440944,
  2: 0.527428949107733,
  3: 0.5263852242744064,
  4: 0.528127068166777,
  5: 0.5260382333553065,
  6: 0.525691699604743,
  7: 0.525691699604743,
  8: 0.5253456221198156}}

In [5]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")


def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


In [6]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: x
print(text_pipeline("dreamy under water coarse sound"))

[207, 456, 0, 0, 7]


In [7]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64) #may cause problems
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


#train_iter = AG_NEWS(split="train")
train_iter = ShardingFilter(train_data)
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

from torch import nn


class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc1 = nn.Linear(embed_dim, 128)
        self.fc2 = nn.Linear(128, num_class)
        self.sigmoid = nn.Sigmoid()
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        fc1 = self.fc1(embedded)
        fc2 = self.fc2(fc1)
        return self.sigmoid(fc2)

In [8]:
num_class = 9
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
print(model)

TextClassificationModel(
  (embedding): EmbeddingBag(1083, 64, mode='mean')
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=9, bias=True)
  (sigmoid): Sigmoid()
)


In [9]:
from tensorflow.keras import backend as K

Wp = class_weights['positive_weights']
Wn = class_weights['negative_weights']
print(Wp)
print(Wn)
class CrossEntropyLossManual:
    """
    y0 is the vector with shape (batch_size,C)
    x shape is the same (batch_size), whose entries are integers from 0 to C-1
    """

    def __init__(self, ignore_index=-100) -> None:
        self.ignore_index=ignore_index
    '''
    def __call__(self, y0, x):
        loss = 0.
        n_batch, n_class = y0.shape
        for y1, x1 in zip(y0, x):
            print(y1, x1)
            class_index = int(x1.item())
            if class_index == self.ignore_index:
                n_batch -= 1
                continue
            loss = loss + torch.log(torch.exp(y1[class_index])/(torch.exp(y1).sum()))
        loss = - loss/n_batch
        return loss
    '''
    def __call__(self, y0, x):
      loss = float(0)
      for i, key in enumerate(Wp.keys()):
          first_term = Wp[key] * y0[i] * torch.log(x[i] + K.epsilon())
          second_term = Wn[key] * (1 - y0[i]) * torch.log(1 - x[i] + K.epsilon())
          loss -= (first_term + second_term)
      return torch.sum(loss)

{0: 4.534090909090909, 1: 5.541666666666667, 2: 4.807228915662651, 3: 4.9875, 4: 4.694117647058824, 5: 5.050632911392405, 6: 5.115384615384615, 7: 5.115384615384615, 8: 5.181818181818182}
{0: 0.5291777188328912, 1: 0.5236220472440944, 2: 0.527428949107733, 3: 0.5263852242744064, 4: 0.528127068166777, 5: 0.5260382333553065, 6: 0.525691699604743, 7: 0.525691699604743, 8: 0.5253456221198156}


In [10]:
def f1_score(y_true, y_logit):
    '''
    Calculate F1 score
    y_true: true value
    y_logit: predicted value
    '''

    true_positives = torch.sum(torch.round(torch.clip(y_true * y_logit, 0, 1)))
    possible_positives = torch.sum(torch.round(torch.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    predicted_positives = torch.sum(torch.round(torch.clip(y_logit, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return (2 * precision * recall) / (precision + recall + K.epsilon())


In [13]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label == label).sum().item() #Can check the first 9 for acc
        #total_acc += f1_score(label,predicted_label)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| f1 score {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label == label).sum().item()
            #total_acc += f1_score(label,predicted_label)
            total_count += label.size(0)
    return total_acc

In [16]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchdata.datapipes.iter import IterableWrapper, Mapper

# Hyperparameters
EPOCHS = 100  # epoch
LR = 5  # learning rate
BATCH_SIZE = 8  # batch size for training

#criterion = CrossEntropyLossManual()
criterion = nn.MultiLabelMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = ShardingFilter(train_data)
test_iter =  ShardingFilter(test_data)
#print(train_iter)

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
#print(train_dataloader)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  0.10s | valid accuracy    2.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  0.09s | valid accuracy    2.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.09s | valid accuracy    2.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.10s | valid accuracy    2.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.10s | valid accuracy    2.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.09s |

In [18]:
print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print("test accuracy {:8.3f}".format(accu_test))

Checking the results of test dataset.
test accuracy   43.000


In [22]:
pedal_label = {1: "Wah", 2: "Fuzz", 3: "Compressor", 4: "EQ", 5: "Distortion", 6: "Delay", 7: "Tremolo", 8: "Chorus", 9: "Flanger"}


def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output


ex_text_str = "swirling happy sensual fuzzy"

model = model.to("cpu")

print(torch.round(predict(ex_text_str, text_pipeline)))

tensor([[1., 1., 0., 0., 0., 0., 0., 0., 0.]])
